In [1]:
import numpy
from scipy import stats
import scipy.special

В качестве априорных распределений возмем сопряженные. Т.е. для $Bin(m,p)$ возмем $Beta(\alpha, \beta)$, для $Pois(\lambda)$ - $\Gamma(a, \theta)$. Тогда

$$ K = \frac{\int\limits_0^1 (\prod\limits_{i=1}^nC_m^{X_i}p^{X_i}(1-p)^{m-X_i})\frac{p^{\alpha}(1-p)^{\beta}}{B(\alpha, \beta)}dp}{\int\limits_0^{+\infty}(\prod\limits_{i=1}^ne^{-\lambda}\frac{\lambda^{X_i}}{X_i!})\frac{\theta^a\lambda^{\alpha-1}}{\Gamma(a)}e^{-\theta\lambda}I(\lambda > 0)d\lambda}=\frac{\left(\prod\limits_{i=1}^nC_m^{X_i}\right)\frac{B\left(\alpha + \sum\limits_{i=1}^nX_i,\ \beta + nm - \sum\limits_{i=1}^nX_i\right)}{B(\alpha, \beta)}}{\left(\prod\limits_{i=1}^n\frac{1}{X_i!}\right)\frac{\theta^{a}\Gamma\left(a + \sum\limits_{i=1}^nX_i\right)}{\Gamma(a)(\theta + n)^{\left(\alpha + \sum\limits_{i=1}^nX_i\right)}}}=$$
$$=\left(\prod\limits_{i=1}^nC_m^{X_i}X_i!\right)\frac{B\left(\alpha + \sum\limits_{i=1}^nX_i,\ \beta + nm - \sum\limits_{i=1}^nX_i\right)\Gamma(\alpha)(\theta + n)^{\left(a + \sum\limits_{i=1}^nX_i\right)}}{B(\alpha, \beta)\theta^{a}\Gamma\left(\alpha + \sum\limits_{i=1}^nX_i\right)}$$

Чтобы значения были не слишком большими, посчитаем $\ln(K)$

$$\ln(K)=\sum\limits_{i=1}^n\Bigl((\ln(C_m^{X_i})+\ln(X_i!)\Bigr) +
\ln\Biggl(B\Bigl(\alpha + \sum\limits_{i=1}^nX_i,\ \beta + nm - \sum\limits_{i=1}^nX_i\Bigr)\Biggr) +
\ln\Bigl(\Gamma(a)\Bigr) +$$
$$+\Bigl(\alpha + \sum\limits_{i=1}^nX_i\Bigr)\ln\Bigl(\theta + n\Bigr) -
\ln\Bigl(B(\alpha, \beta)\Bigr)+
\alpha\ln(\theta)+\ln\Bigl(\Gamma\Bigl(\alpha + \sum\limits_{i=1}^nX_i\Bigr)\Bigr)$$

In [2]:
# Выбираем какие-нибудь параметры для распределения соответствующего H_0
m = 10
p = 1 / 10
lam = 3
N = 1000

# Выбираем какие-нибудь параметры априорных рапределений
alpha = 1
beta = 1
theta = 1

In [3]:
# Для оценки квантилей генерируем большое число выборок

bootstrap_size = 1000
bin_bootstraps = []
for n in {50, 100, 200}:
    bin_bootstraps.append([stats.binom.rvs(m, p, size=n) for i in range(bootstrap_size)])
bin_bootstraps = numpy.array(bin_bootstraps)

In [4]:
def get_K(sample):
    return numpy.exp(
        numpy.sum([numpy.log(scipy.special.comb(m, x)) + scipy.special.gammaln(x + 1) for x in sample]) +
        scipy.special.betaln(alpha + numpy.sum(sample), beta + m * N - numpy.sum(sample)) +
        scipy.special.gammaln(alpha) +
        (alpha + numpy.sum(sample)) * numpy.log(theta + N) -
        scipy.special.betaln(alpha, beta) -
        alpha * numpy.log(theta) -
        scipy.special.gammaln(alpha + numpy.sum(sample))
    )

In [5]:
percentilies = []

for alpha in {0.1, 0.05, 0.01}:
    for bootstrap in bin_bootstraps:
        percentile = numpy.percentile([get_K(sample) for sample in bootstrap], alpha)
        percentilies.append((alpha, len(bootstrap[0]), percentile))
        
        print("alpha={}, n={}, u={}".format(alpha, len(bootstrap[0]), percentile))

alpha=0.05, n=200, u=1.9045554472061642e-06
alpha=0.05, n=50, u=0.0022304728176510465
alpha=0.05, n=100, u=0.00014354185419302001
alpha=0.1, n=200, u=2.1627530389680354e-06
alpha=0.1, n=50, u=0.002166096429900412
alpha=0.1, n=100, u=0.0001451725244874687
alpha=0.01, n=200, u=1.61461814620288e-06
alpha=0.01, n=50, u=0.0022681626462545757
alpha=0.01, n=100, u=0.00014002872141239471


Для каждого уровня значимости и для каждого $n$ с помощью посчитанных квантилей получаем критерий вида $\{K>u\}$